涉及寻找最优权重参数的最优化方法、权重参数的初始值，超参数的设定方法等，
为了应对过拟合，学习权值衰减，Dropout等正则化方法，并进行实现，
学习Batch Normalization 方法，通过这些方法可以高效的进行神经网络的学习，提高识别精度。

1、参数的更新
2、权重的1初始值
3、Batch Normalization
4、正则化
5、超参数的验证


参数的更新

神经网络的学习目的是找到使损失函数的值尽可能小的参数，这是寻找最优参数的问题，解决这个问题的过程称为最优化。

随机梯度下降法（stochastic gradient descent），简称SGD
    
$$
W = W - \eta \frac{\partial L}{\partial W}
$$
式中，$W$是权重参数，$\eta$是学习率，$\frac{\partial L}{\partial W}$是损失函数关于权重参数的导数。
SGD是朝着梯度方向只前进一定距离的简单方法

In [ ]:
class SGD:
    def __init__(self, lr=0.01):
        self.lr = lr              #学习率
        
    def update(self, params, grads):
        for key in params.keys():
            params[key] -= self.lr * grads[key]

update（params,grads） 方法在SGD中被反复调用，参数params 和 grads 是字典型变量，按params['W1']、grads['W1']的形式，分别保存了权重参数和它们的梯度。使用SGD这个类，可以像下面这样更新权重参数。
```
# 假设神经网络有2层
network = TwoLayerNet(...)
optimizer = SGD()           #参数的更新由optimizer负责完成，需要做的只是将参数和梯度的信息传给optimizer

for i in range(10000):
    ...
    # 进行mini-batch学习
    x_batch, t_batch = get_mini_batch(...)  # mini-batch
    grads = network.gradient(x_batch, t_batch)
    optimizer.update(network.params, grads)
    ...
```

通过单独实现进行最优化的类，功能的模块化变得简单，如果要实现另外一个最优化Momentum，它同样会实现成拥有update(params, grads)这个共同方法的形式。这样一来，只需要将optimizer = SGD()这一语句换成optimizer = Momentum()，就可以从SGD切换为Momentum。

SGD的缺点：
- 学习率lr是固定值，不能自动调整。
- 不能保证到达最小值。
- 可能被困在鞍点（saddle point）。

为了改正SGD 将介绍Momentum、AdaGrad、Adam 这3种方法来取代SGD

Momentum 动量法
公式
$$
v = \alpha v - \eta \frac{\partial L}{\partial W} \\
W = W + v
$$
其中，$v$是速度，$\alpha$是动量系数，$\eta$是学习率，$\frac{\partial L}{\partial W}$是损失函数关于权重$W$的梯度。


In [ ]:
class Momentum:
    def __init__(self, lr=0.01, momentum=0.9):
        self.lr = lr
        self.momentum = momentum
        self.v = None

    def update(self, params, grads):
        if self.v is None:
            self.v = {}
            for key, val in params.items():
                self.v[key] = np.zeros_like(val)
        for key in params.keys():
            self.v[key] = self.momentum*self.v[key] - self.lr*grads[key]
            params[key] += self.v[key]

实例变量v会保存物体的速度。初始化时，v中什么都不保存，但当第一次调用update()时，v会以字典型变量的形式保存与参数结构相同的数据。

Adagrad 

Adagrad 是一个针对学习率进行自适应调整的方法。学习率过小，会导致学习花费过多时间，学习率过大，会导致学习发散而不能正确进行。

在关于学习率的有效技巧中，有一种被称为学习率衰减（learning rate decay）的方法。学习率衰减是指随着学习的进行，学习率逐渐减小的方法。这样做的目的是为了在学习开始时，使学习率较大，从而快速学习；而在学习结束时，使学习率较小，从而使学习更加稳定,逐渐减小学习率的想法，相当于将全体参数的学习率值一起降低而AdaGrad进一步发展了这个想法，针对“一个一个”的参数，赋予其“定制”的值。

AdaGrad会为参数的每个元素适当地调整学习率，与此同时进行学习（AdaGrad的Ada来自英文单词Adaptive，即“适当的”的意思）。下面，让我们用数学式表示AdaGrad的更新方法。

公式
$$
h = h + \frac{\partial L}{\partial W} \odot \frac{\partial L}{\partial W} \\
W = W - \eta \frac{1}{\sqrt{h}} \odot \frac{\partial L}{\partial W}
$$
其中，$h$是累加的梯度的平方和，$\eta$是学习率，$\odot$是按元素相乘的运算符。
变量h保存了以前所有梯度值的平方和，参数的元素中变动较大（被大幅更新）的元素的学习率将变小。也就是说，可以按参数的元素进行学习率衰减，使变动大的参数的学习率逐渐减小。


In [ ]:
class Adagrad:
    def __init__(self, lr=0.01):
        self.lr = lr
        self.h = None
    def update(self, params, grads):
        if self.h is None:
            self.h = {}
            for key, val in params.items():
                self.h[key] = np.zeros_like(val)
        for key in params.keys():
            self.h[key] += grads[key] * grads[key]
            params[key] -= self.lr * grads[key] / (np.sqrt(self.h[key]) + 1e-7)

这里需要注意的是，最后一行加上了微小值1e-7。这是为了防止当self.h[key]中有0时，将0用作除数的情况。在很多深度学习的框架中，这个微小值也可以设定为参数，但这里我们用的是1e-7这个固定值。

Adam
公式
$$
m_t = \beta_1 m_{t-1} + (1 - \beta_1) \triangledown \theta_t
$$
$$
v_t = \beta_2 v_{t-1} + (1 - \beta_2) \triangledown \theta_t^2
$$
$$
\theta_t = \theta_{t-1} - \frac{\alpha \sqrt{1 - \beta_2^t}}{1 - \beta_1^t} \frac{m_t}{\sqrt{v_t} + \epsilon}
$$


Momentum参照小球在碗中滚动的物理规则进行移动，AdaGrad为参数的每个元素适当地调整更新步伐。如果将这两个方法融合在一起会怎么样?
它的理论有些复杂，直观地讲，就是融合了Momentum和AdaGrad的方法。通过组合前面两个方法的优点，有望实现参数空间的高效搜索。此外，进行超参数的“偏置校正”也是Adam的特征。


Adam会设置3个超参数。一个是学习率（论文中以α出现），另外两个是一次momentum系数β1和二次momentum系数β2。根据论文，标准的设定值是β1为0.9，β2为0.999。设置了这些值后，大多数情况下都能顺利运行。

In [ ]:
class Adam:
    def __init__(self, lr=0.001, beta1=0.9, beta2=0.999):
        self.lr = lr
        self.beta1 = beta1
        self.beta2 = beta2
        self.iter = 0
        self.m = None
        self.v = None
    def update(self, params, grads):
        if self.m is None:
            self.m, self.v = {}, {}
            for key, val in params.items():
                self.m[key] = np.zeros_like(val)
                self.v[key] = np.zeros_like(val)
        self.iter += 1
        lr_t = self.lr * np.sqrt(1.0 - self.beta2**self.iter) / (1.0 - self.beta1**self.iter)
        for key in params.keys():
            self.m[key] += (1 - self.beta1) * (grads[key] - self.m[key])
            self.v[key] += (1 - self.beta2) * (grads[key]**2 - self.v[key])
